In [ ]:
import re
import pandas as pd

In [ ]:
#Task 1
with open("input.txt") as f:
    rawText = f.read().strip()
    splittedText = re.split("[a-z\-\s:]{2,}", rawText)[1:]
seeds = list(map(lambda x: int(x), splittedText[0].split(" ")))
transformers = splittedText[1:]
transformers = list(map(lambda x: x.split('\n'), transformers))
transformers = list(map(lambda x: list(map(lambda y: list(map(lambda z: int(z), y.split(" "))), x)), transformers))
seeds, transformers

def getLocationsForSeeds(seeds, transformers):
    locations = []
    for seed in seeds:
        currentValue = seed
        for transformer in transformers:
            for mapping in transformer:
                if currentValue >= mapping[1] and currentValue <= mapping[1]+mapping[2]-1:
                    currentValue = mapping[0]+(currentValue-mapping[1])
                    break
        locations.append(currentValue)
    return locations
min(getLocationsForSeeds(seeds, transformers))

In [ ]:
#Task 2
chunks = [seeds[x:x+2] for x in range(0, len(seeds), 2)]
seedRanges = [(x[0], x[0]+x[1]-1) for x in chunks]
locationRanges = []

for seedRange in seedRanges:
    currentRanges = [seedRange]
    for transformer in transformers:
        rangesToBeProcessed = [x for x in currentRanges]
        postTransformerList = []
        while len(rangesToBeProcessed) > 0:
            evaluatedRange = rangesToBeProcessed.pop()
            rangeWasMapped = False
            for mapping in transformer:
                sourceRange = (mapping[1], mapping[1]+mapping[2]-1)
                appliedDifference = mapping[1]-mapping[0]

                #evaluated range fully wrapped by source range
                if evaluatedRange[0] >= sourceRange[0] and evaluatedRange[1]<=sourceRange[1]:
                    postTransformerList.append((evaluatedRange[0]-appliedDifference, evaluatedRange[1]-appliedDifference))
                    #print(str(evaluatedRange) + " is fully wrapped by " + str(sourceRange))
                    rangeWasMapped = True
                    break
                #evaluated range left bound outside right bound inside
                elif evaluatedRange[0] < sourceRange[0] and evaluatedRange[1] >= sourceRange[0] and evaluatedRange[1] <= sourceRange[1]:
                    rangesToBeProcessed.append((evaluatedRange[0], sourceRange[0]-1))
                    postTransformerList.append((sourceRange[0]-appliedDifference, evaluatedRange[1]-appliedDifference))
                    #print(str(evaluatedRange) + " overlays on the left " + str(sourceRange)+":"+str(rangesToBeProcessed[-1])+str(postTransformerList[-1]))
                    rangeWasMapped = True
                    break
                #evaluated range right bound outside, left bound inside
                elif evaluatedRange[0] >= sourceRange[0] and evaluatedRange[0] <= sourceRange[1] and evaluatedRange[1] > sourceRange[1]:
                    rangesToBeProcessed.append((sourceRange[1]+1, evaluatedRange[1]))
                    postTransformerList.append((evaluatedRange[0]-appliedDifference, sourceRange[1]-appliedDifference))
                    #print(str(evaluatedRange) + " overlays on the right " + str(sourceRange))
                    rangeWasMapped = True
                    break
                #evaluated range wraps source range
                elif evaluatedRange[0] < sourceRange[0] and evaluatedRange[1] > sourceRange[1]:
                    postTransformerList.append((sourceRange[0]-appliedDifference, sourceRange[1]-appliedDifference))
                    rangesToBeProcessed.append((evaluatedRange[0], sourceRange[0]-1))
                    rangesToBeProcessed.append((sourceRange[1]+1, evaluatedRange[1]))
                    #print(str(evaluatedRange) + " wraps " + str(sourceRange))
                    rangeWasMapped = True
                    break
            if not rangeWasMapped: 
                #print(str(evaluatedRange) + " did not match anything in that transfromer " + str(transformer))
                postTransformerList.append(evaluatedRange)
        currentRanges = postTransformerList
    locationRanges+=currentRanges
minLocation = min(list(map(lambda x: x[0], locationRanges)))
minLocation 
